# Predict VoxCeleb utterance info from SSL representations

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.chdir('../..')
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [2]:
from notebooks.notebooks_utils import (
    load_models,
    evaluate_models,
    create_metrics_df
)

from sslsv.evaluations.CosineSVEvaluation import CosineSVEvaluation, CosineSVEvaluationTaskConfig

In [18]:
models = load_models(
    [
        './exps/old/vox2_ddp_sntxent_s=30_m=0/config.yml',
        './exps/old/vox2_ddp_sntxent_s=30_m=0.1/config.yml'
    ],
    override_names={
        'exps/old/vox2_ddp_sntxent_s=30_m=0'   : 'simclr',
        'exps/old/vox2_ddp_sntxent_s=30_m=0.1' : 'simclr_am'
    }
)

In [19]:
evaluate_models(models, CosineSVEvaluation, CosineSVEvaluationTaskConfig())

Extracting test embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:21<00:00,  3.42it/s]


In [20]:
from sklearn.linear_model import LogisticRegression

def fit_mlp_on_representations(model, y_key_pos):
    keys = list(model['embeddings'].keys())
    
    X = [model['embeddings'][key][0].numpy() for key in keys]
    if y_key_pos is None:
        y = keys
    else:
        y = [key.split('/')[y_key_pos] for key in keys]
    
    clf = LogisticRegression()
    clf.fit(X, y)
    
    print(f'Accuracy: {clf.score(X, y)}')
    
    return clf, X, y

## Speaker ID

In [21]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=1)

Accuracy: 0.994052676295667


In [22]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=1)

Accuracy: 0.9966015293118097


## Video ID

In [23]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=2)

Accuracy: 0.6053525913338997


In [24]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=2)

Accuracy: 0.6295666949872557


## Segment ID

In [25]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=3)

Accuracy: 0.1671622769753611


/work2/home/ing2/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=3)

Accuracy: 0.1620645709430756


/work2/home/ing2/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Sample ID

In [27]:
_ = fit_mlp_on_representations(models['simclr'], y_key_pos=None)

Accuracy: 0.9961767204757859


In [28]:
_ = fit_mlp_on_representations(models['simclr_am'], y_key_pos=None)

Accuracy: 0.9963891248937978
